In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 250 kB 30.8 MB/s 
     |████████████████████████████████| 1.9 MB 50.7 MB/s 
     |████████████████████████████████| 10.3 MB 63.2 MB/s 
     |████████████████████████████████| 1.3 MB 60.1 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 5.5 MB 53.1 MB/s 
     |████████████████████████████████| 7.6 MB 56.1 MB/s 
     |████████████████████████████████| 451 kB 68.1 MB/s 
     |████████████████████████████████| 182 kB 70.3 MB/s 
     |████████████████████████████████| 168 kB 68.4 MB/s 
     |████████████████████████████████| 182 kB 61.4 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 75.4 MB/s 
     |████████████████████████████████| 166 kB 79.1 MB/s 
     |████████████████████████████████| 166 kB 80.1 MB/s 
     |████████████████████████████

# Import the train and test dataset 

In [3]:
!wget https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
!wget https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip

--2022-11-25 09:41:12--  https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3950125 (3.8M) [application/zip]
Saving to: ‘e2eqa-train+public_test-v1.zip’

e2eqa-train+public_ 100%[===================>]   3.77M  12.2MB/s    in 0.3s    

2022-11-25 09:41:13 (12.2 MB/s) - ‘e2eqa-train+public_test-v1.zip’ saved [3950125/3950125]

--2022-11-25 09:41:13--  https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 417016977 (398M) [application/zip]
Saving to: ‘wikipedia_20220620_cleaned.zip’

wikipedia_20220620_

In [4]:
!wget https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt

--2022-11-25 09:41:20--  https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20475 (20K) [text/plain]
Saving to: ‘vietnamese-stopwords-dash.txt’

vietnamese-stopword 100%[===================>]  20.00K  --.-KB/s    in 0s      

2022-11-25 09:41:20 (117 MB/s) - ‘vietnamese-stopwords-dash.txt’ saved [20475/20475]



In [5]:
!unzip -q /content/e2eqa-train+public_test-v1.zip
!unzip -q /content/wikipedia_20220620_cleaned.zip

In [6]:
!pip install pyvi
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.5 MB 17.5 MB/s 
     |████████████████████████████████| 965 kB 45.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import json

In [8]:
with open(r"/content/drive/MyDrive/zaloAI/train.json", "r") as read_file:
    train = json.load(read_file)

In [12]:
with open(r"/content/drive/MyDrive/zaloAI/dataset/test.json", "r") as read_file:
    test = json.load(read_file)

In [13]:
test

[{'context': 'Trụ sở Liên Hiệp Quốc được đặt trong lãnh phận quốc tế tại Manhattan , Thành phố New York , Hoa Kỳ .',
  'qas': [{'id': '4804fb60646161723f3b31c1bcccce5d',
    'is_impossible': False,
    'question': 'Trụ sở chính của Liên Hiệp Quốc đặt trong lãnh phận quốc tế tại quận nào của thành phố New York',
    'answers': [{'text': 'tại Manhattan , Thành phố New York , Hoa Kỳ .',
      'answer_start': 55}]}]},
 {'context': 'Và năm 2015 tác giả Eiichiro Oda và bộ manga One Piece được ghi nhận vào Sách Kỷ lục Guinness khi là bộ truyện tranh do đơn tác giả sáng tác có số bản in nhiều nhất thế giới . Hiện tại , One Piece đang nắm giữ kỷ lục là bộ manga bán chạy nhất trong lịch sử .',
  'qas': [{'id': '287d81231704ddeb428d660447b6f68d',
    'is_impossible': False,
    'question': 'Bộ manga One Piece do ai sáng tác',
    'answers': [{'text': 'Eiichiro Oda', 'answer_start': 20}]}]},
 {'context': 'Rách màng trinh . Theo một khảo sát , chỉ có 43% phụ nữ bị chảy máu khi quan hệ tình dục lần 

#Import the model and start configure it to train 

In [14]:
import logging


from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs


In [15]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"


In [16]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.use_cuda = False
model_args.evaluate_during_training = False 
model_args.n_best_size=3
model_args.num_train_epochs=3

In [17]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 114,
    "num_train_epochs": 3,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 64,
    "eval_batch_size": 32,
    # "config": {
    #     "output_hidden_states": True
    # }
}


# apply the configuration and train 

In [19]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args , use_cuda = False
)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [20]:
### Remove output folder
!rm -rf outputs

In [ ]:
# Train the model
model.train_model(train, eval_data=test)

add example index and unique id: 100%|██████████| 3742/3742 [00:00<00:00, 667733.91it/s]


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Running Epoch 0 of 3:   0%|          | 0/61 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 3742/3742 [00:13<00:00, 282.26it/s]

add example index and unique id: 100%|██████████| 3742/3742 [00:00<00:00, 463610.96it/s]


Running Evaluation:   0%|          | 0/121 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/61 [00:00<?, ?it/s]

In [ ]:



!mv outputs /content/drive/MyDrive/zaloAI/BERT
!mv runs /content/drive/MyDrive/zaloAI/BERT
!mv wandb /content/drive/MyDrive/zaloAI/BERT

In [ ]:
# Evaluate the model
result, texts = model.eval_model(test)

In [ ]:
%cd /content/drive/MyDrive/zaloAI/BERT

/content/drive/MyDrive/zaloAI/BERT


In [ ]:
# Make predictions with the model
# Make predictions with the model
to_predict = [
    {
        "context": "Washington Irving là người tạo ra cỗ xe ngọt ngào của Santa ",
        "qas": [
            {
                "question": "Ai là người tạo ra cỗ xe ngọt ngào của Santa",
                "id": "0",
            }
        ],
    }
]

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 9467.95it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['xe ngọt ngào củ']}]


In [ ]:
# load zalo AI cleaned wiki 
!wget  https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip

--2022-11-23 06:36:24--  https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 417016977 (398M) [application/zip]
Saving to: ‘wikipedia_20220620_cleaned.zip.1’

wikipedia_20220620_ 100%[===================>] 397.70M  10.2MB/s    in 38s     

2022-11-23 06:37:04 (10.3 MB/s) - ‘wikipedia_20220620_cleaned.zip.1’ saved [417016977/417016977]

